In [272]:
#トラカンデータの距離案分()
import numpy as np
import psycopg2
import datetime
import pandas as pd

In [359]:
#計測地点のosm_idを指定
start_osm = 51396950
mid_osm = []
end_osm =  51396950
start_mesh = 50325642222
end_mesh = 50325632444

In [360]:
connection = psycopg2.connectconnection = psycopg2.connect(host="localhost", database="analysisdb", user="postgres",password="setouchi1311")

In [361]:
getdbname = "kddi_00_new_param"
dbname = "kddi_trkn00_monthly_anbun"
trknName = "trkn_00_grp"
sttime = "2022-04-01 00:00:00"

In [362]:
for i in range(30):
    dte = datetime.datetime.strptime(sttime, '%Y-%m-%d %H:%M:%S')
    if i != 0:
        dte = dte + datetime.timedelta(days=i)
    for j in range(24):
        timestr = dte.strftime('%H:%M:%S')
        #特定日付のKDDIメッシュ情報を全取得
        cur =connection.cursor()
        sql = "SELECT mesh, tbran,pointdata.latitude,pointdata.longitude FROM " + getdbname + " "
        sql += "INNER JOIN pointdata ON mesh = cast(pointdata.grid_code AS BIGINT)  "
        sql += "WHERE times = '"+str(dte)+"' AND pointdata.grid_code != '0'  ORDER BY mesh "
        cur.execute(sql)
        kddiDf = pd.DataFrame.from_records(cur.fetchall(), columns=["mesh","tbran","latitude","longitude"])
        cur.close()

        one_target_list = []
        one_target_list.append(start_mesh)
        one_target_list.append(end_mesh)
        two_target_list = []
        two_target_list.append(start_osm)
        if len(mid_osm) != 0:
            for target in mid_osm:
                two_target_list.append(target)
        if start_osm != end_osm:
            two_target_list.append(end_osm)

        cur =connection.cursor()
        sql = "SELECT latitude,longitude FROM pointdata WHERE pointdata.grid_code = '"+str(start_mesh)+"'"
        cur.execute(sql)
        for row in cur:
            stlat = row[0]
            stlng = row[1]
        sql = "SELECT latitude,longitude FROM pointdata WHERE pointdata.grid_code = '"+str(end_mesh)+"'"
        cur.execute(sql)
        for row in cur:
            edlat = row[0]
            edlng = row[1]
        cur.close()

        if stlat > edlat:
            wklat = stlat
            stlat = edlat
            edlat = wklat
        if stlng > edlng:
            wklng = stlng
            stlng = edlng
            edlng = wklng

        print(str(stlat) + ":" + str(edlat) + ":" + str(stlng) + ":" + str(stlng))

        keylist = []
        cnt = 0
        for item in two_target_list:
            sql = "SELECT a.grid_code,a.latitude,a.longitude"
            sql += " FROM pointdata a, roadmap r"
            sql += " WHERE osm_id = "+ str(item) +" and ST_Distance(a.geom, r.geom) < 0.001"
            sql += " order by ST_Distance(a.geom, r.geom);"
            cur =connection.cursor()
            cur.execute(sql)
            childlist = []
            for row in cur:
                if row[1] >= stlat and row[1] <= edlat and row[2] >= stlng and row[2] <= edlng:
                    childlist.append([row[0], row[1], row[2]])
                #print(row[0])
            worklist = [item, childlist]
            keylist.append(worklist)
            cnt = cnt + 1
            cur.close()
        #特定日付のトラカンデータを全取得 > 緯度経度
        cur =connection.cursor()
        #sql = "SELECT trakanmap.gridcode , 断面交通量,pointdata.latitude,pointdata.longitude FROM trakan2204 "
        #sql += "INNER JOIN trakanmap ON trakan2204.計測地点番号 = trakanmap.地点番号 "
        #sql += "INNER JOIN pointdata ON trakanmap.gridcode = pointdata.grid_code "
        #sql += "WHERE 時刻 = '"+str(dte)+"' AND (trakanmap.gridcode = '"+str(start_mesh)+"' or trakanmap.gridcode = '"+str(end_mesh)+"') "
        #sql += "ORDER BY trakanmap.gridcode"
        sql = "SELECT grid_code, tbran, latitude, longitude FROM " + trknName  + " "
        sql += "INNER JOIN pointdata ON mesh = cast(grid_code as bigint) "
        sql += "WHERE times = '"+timestr+"' AND (mesh = '"+str(start_mesh)+"' or mesh = '"+str(end_mesh)+"') "
        sql += "ORDER BY mesh"
        #print(sql)
        cur.execute(sql)
        trknDf = pd.DataFrame(cur.fetchall(), columns=["gridcode","danmen","latitude","longitude"])
        cur.close()
        trknDf
        #keylist

        from geopy.distance import geodesic

        #データが存在する道路IDごとに交通量を平均化して取得する
        lstItems = []
        cntDt = 0
        wks = ""
        wknp=""
        find = 0

        trknStParam = trknDf.query('gridcode == "'+str(start_mesh) + '"')
        trknEdParam = trknDf.query('gridcode == "'+str(end_mesh) + '"')
        trknStParam

        if len(trknStParam) == 0:
            print(str(start_mesh) + " mesh empty!")
            dte = dte + datetime.timedelta(minutes=15) 
            continue
        elif len(trknStParam) == 1:
            trknSt = trknStParam['danmen'].iloc[-1]
        else:
            trknSt = trknStParam['danmen'].iloc[0] + trknStParam['danmen'].iloc[1]

        if len(trknEdParam) == 0:
            print(str(end_mesh) + " mesh empty!")
            dte = dte + datetime.timedelta(minutes=15) 
            continue
        elif len(trknEdParam) == 1:
            trknEd = trknEdParam['danmen'].iloc[-1]
        else:
            trknEd = trknEdParam['danmen'].iloc[0] + trknEdParam['danmen'].iloc[1]

        st_station = [trknStParam['latitude'].iloc[-1], trknStParam['longitude'].iloc[-1]]
        ed_station = [trknEdParam['latitude'].iloc[-1], trknEdParam['longitude'].iloc[-1]]

        dis_km = geodesic(st_station, ed_station)
        cur =connection.cursor()
        for row in keylist:
            for meshid in row[1]:
                #路線のメッシュ数分ループ
                print(str(meshid[0]))
                kddiParam = kddiDf.query('mesh == '+str(meshid[0]))
                if len(kddiParam) == 0:
                    print(str(meshid[0]) + " is kddi none!")
                    continue
                #路線のメッシュがkddiに存在したなら
                
                kddi_station = [kddiParam['latitude'].iloc[-1], kddiParam['longitude'].iloc[-1]]
                kddi_km = geodesic(st_station, kddi_station)
                
                if trknSt < trknEd:
                    ret = ((trknEd - trknSt) * (kddi_km / dis_km)) + trknSt
                else:
                    ret = trknSt - ((trknEd - trknSt) * (kddi_km / dis_km))
                print("存在メッシュ：" + str(meshid[0]) + "値" + str(ret) + "距離" + str(kddi_km))
                sql = "INSERT INTO "+dbname+" (times, stmesh, edmesh, mesh, tbran) VALUES ('"+str(dte)+"',"+str(start_mesh)+","  +str(end_mesh)+","  + str(meshid[0]) + ","  +  str(ret) + ") ON CONFLICT DO NOTHING;"
                cur.execute(sql)
                connection.commit()

        sql = "INSERT INTO "+dbname+" (times, stmesh, edmesh, mesh, tbran) VALUES ('"+str(dte)+"',"+str(start_mesh)+","  +str(end_mesh)+","  + str(start_mesh) + ","  + str(trknSt) + ") ON CONFLICT DO NOTHING;"
        cur.execute(sql)
        connection.commit()
        cur.close()
        dte = dte + datetime.timedelta(minutes=15) 
        #print(lstItems)


33.7828125000:33.7838541665:132.786718750:132.786718750
50325642222
存在メッシュ：50325642222値2.21875距離0.0 km
50325632444
存在メッシュ：50325632444値2.8333333333333335距離0.1155400634006377 km
33.7828125000:33.7838541665:132.786718750:132.786718750
50325642222
存在メッシュ：50325642222値2.388888888888889距離0.0 km
50325632444
50325632444 is kddi none!
33.7828125000:33.7838541665:132.786718750:132.786718750
50325642222
存在メッシュ：50325642222値1.95距離0.0 km
50325632444
50325632444 is kddi none!
33.7828125000:33.7838541665:132.786718750:132.786718750
50325642222
存在メッシュ：50325642222値2.025距離0.0 km
50325632444
50325632444 is kddi none!
33.7828125000:33.7838541665:132.786718750:132.786718750
50325642222
存在メッシュ：50325642222値2.206896551724138距離0.0 km
50325632444
50325632444 is kddi none!
33.7828125000:33.7838541665:132.786718750:132.786718750
50325642222
存在メッシュ：50325642222値1.7307692307692308距離0.0 km
50325632444
50325632444 is kddi none!
33.7828125000:33.7838541665:132.786718750:132.786718750
50325642222
存在メッシュ：50325642222値1.9545

33.7828125000:33.7838541665:132.786718750:132.786718750
50325642222
50325642222 is kddi none!
50325632444
存在メッシュ：50325632444値2.1504702194357366距離0.1155400634006377 km
33.7828125000:33.7838541665:132.786718750:132.786718750
50325642222
50325642222 is kddi none!
50325632444
存在メッシュ：50325632444値2.928571428571429距離0.1155400634006377 km
33.7828125000:33.7838541665:132.786718750:132.786718750
50325642222
50325642222 is kddi none!
50325632444
存在メッシュ：50325632444値1.5782145782145782距離0.1155400634006377 km
33.7828125000:33.7838541665:132.786718750:132.786718750
50325642222
50325642222 is kddi none!
50325632444
存在メッシュ：50325632444値2.4687791239515375距離0.1155400634006377 km
33.7828125000:33.7838541665:132.786718750:132.786718750
50325642222
50325642222 is kddi none!
50325632444
存在メッシュ：50325632444値2.3046251993620412距離0.1155400634006377 km
33.7828125000:33.7838541665:132.786718750:132.786718750
50325642222
50325642222 is kddi none!
50325632444
50325632444 is kddi none!
33.7828125000:33.7838541665:132.78

33.7828125000:33.7838541665:132.786718750:132.786718750
50325642222
50325642222 is kddi none!
50325632444
存在メッシュ：50325632444値3.765597147950089距離0.1155400634006377 km
33.7828125000:33.7838541665:132.786718750:132.786718750
50325642222
50325642222 is kddi none!
50325632444
存在メッシュ：50325632444値2.916666666666667距離0.1155400634006377 km
33.7828125000:33.7838541665:132.786718750:132.786718750
50325642222
50325642222 is kddi none!
50325632444
存在メッシュ：50325632444値2.6090225563909772距離0.1155400634006377 km
33.7828125000:33.7838541665:132.786718750:132.786718750
50325642222
50325642222 is kddi none!
50325632444
存在メッシュ：50325632444値2.916666666666667距離0.1155400634006377 km
33.7828125000:33.7838541665:132.786718750:132.786718750
50325642222
50325642222 is kddi none!
50325632444
存在メッシュ：50325632444値2.67966991747937距離0.1155400634006377 km
33.7828125000:33.7838541665:132.786718750:132.786718750
50325642222
50325642222 is kddi none!
50325632444
存在メッシュ：50325632444値1.8911111111111112距離0.1155400634006377 km
33.

50325642222
存在メッシュ：50325642222値2.0距離0.0 km
50325632444
50325632444 is kddi none!
33.7828125000:33.7838541665:132.786718750:132.786718750
50325642222
存在メッシュ：50325642222値1.967741935483871距離0.0 km
50325632444
50325632444 is kddi none!
33.7828125000:33.7838541665:132.786718750:132.786718750
50325642222
存在メッシュ：50325642222値1.5555555555555556距離0.0 km
50325632444
50325632444 is kddi none!
33.7828125000:33.7838541665:132.786718750:132.786718750
50325642222
存在メッシュ：50325642222値2.225806451612903距離0.0 km
50325632444
50325632444 is kddi none!
33.7828125000:33.7838541665:132.786718750:132.786718750
50325642222
存在メッシュ：50325642222値2.5405405405405403距離0.0 km
50325632444
50325632444 is kddi none!
33.7828125000:33.7838541665:132.786718750:132.786718750
50325642222
存在メッシュ：50325642222値2.3142857142857145距離0.0 km
50325632444
50325632444 is kddi none!
33.7828125000:33.7838541665:132.786718750:132.786718750
50325642222
存在メッシュ：50325642222値2.909090909090909距離0.0 km
50325632444
50325632444 is kddi none!
33.7828125

33.7828125000:33.7838541665:132.786718750:132.786718750
50325642222
存在メッシュ：50325642222値3.5106382978723403距離0.0 km
50325632444
存在メッシュ：50325632444値4.143725575336518距離0.1155400634006377 km
33.7828125000:33.7838541665:132.786718750:132.786718750
50325642222
存在メッシュ：50325642222値4.0588235294117645距離0.0 km
50325632444
存在メッシュ：50325632444値4.9390756302521距離0.1155400634006377 km
33.7828125000:33.7838541665:132.786718750:132.786718750
50325642222
存在メッシュ：50325642222値2.21875距離0.0 km
50325632444
存在メッシュ：50325632444値2.8333333333333335距離0.1155400634006377 km
33.7828125000:33.7838541665:132.786718750:132.786718750
50325642222
50325642222 is kddi none!
50325632444
存在メッシュ：50325632444値2.813953488372093距離0.1155400634006377 km
33.7828125000:33.7838541665:132.786718750:132.786718750
50325642222
50325642222 is kddi none!
50325632444
存在メッシュ：50325632444値2.0227272727272725距離0.1155400634006377 km
33.7828125000:33.7838541665:132.786718750:132.786718750
50325642222
50325642222 is kddi none!
50325632444
存在メッシュ：50325632

...

33.7828125000:33.7838541665:132.786718750:132.786718750
50325642222
存在メッシュ：50325642222値1.9473684210526316距離0.0 km
50325632444
存在メッシュ：50325632444値2.6090225563909772距離0.1155400634006377 km
33.7828125000:33.7838541665:132.786718750:132.786718750
50325642222
存在メッシュ：50325642222値2.0距離0.0 km
50325632444
50325632444 is kddi none!
33.7828125000:33.7838541665:132.786718750:132.786718750
50325642222
50325642222 is kddi none!
50325632444
50325632444 is kddi none!
33.7828125000:33.7838541665:132.786718750:132.786718750
50325642222
50325642222 is kddi none!
50325632444
50325632444 is kddi none!
33.7828125000:33.7838541665:132.786718750:132.786718750
50325642222
50325642222 is kddi none!
50325632444
50325632444 is kddi none!
33.7828125000:33.7838541665:132.786718750:132.786718750
50325642222
50325642222 is kddi none!
50325632444
50325632444 is kddi none!
33.7828125000:33.7838541665:132.786718750:132.786718750
50325642222
50325642222 is kddi none!
50325632444
50325632444 is kddi none!
33.7828125000:33

33.7828125000:33.7838541665:132.786718750:132.786718750
50325642222
50325642222 is kddi none!
50325632444
存在メッシュ：50325632444値3.099949005609383距離0.1155400634006377 km
33.7828125000:33.7838541665:132.786718750:132.786718750
50325642222
50325642222 is kddi none!
50325632444
存在メッシュ：50325632444値3.236414565826331距離0.1155400634006377 km
33.7828125000:33.7838541665:132.786718750:132.786718750
50325642222
50325642222 is kddi none!
50325632444
存在メッシュ：50325632444値3.2898799313893656距離0.1155400634006377 km
33.7828125000:33.7838541665:132.786718750:132.786718750
50325642222
50325642222 is kddi none!
50325632444
存在メッシュ：50325632444値4.143725575336518距離0.1155400634006377 km
33.7828125000:33.7838541665:132.786718750:132.786718750
50325642222
50325642222 is kddi none!
50325632444
存在メッシュ：50325632444値4.9390756302521距離0.1155400634006377 km
33.7828125000:33.7838541665:132.786718750:132.786718750
50325642222
存在メッシュ：50325642222値2.21875距離0.0 km
50325632444
存在メッシュ：50325632444値2.8333333333333335距離0.115540063400637

33.7828125000:33.7838541665:132.786718750:132.786718750
50325642222
存在メッシュ：50325642222値4.0588235294117645距離0.0 km
50325632444
存在メッシュ：50325632444値4.9390756302521距離0.1155400634006377 km
33.7828125000:33.7838541665:132.786718750:132.786718750
50325642222
存在メッシュ：50325642222値2.21875距離0.0 km
50325632444
存在メッシュ：50325632444値2.8333333333333335距離0.1155400634006377 km
33.7828125000:33.7838541665:132.786718750:132.786718750
50325642222
存在メッシュ：50325642222値2.388888888888889距離0.0 km
50325632444
存在メッシュ：50325632444値2.813953488372093距離0.1155400634006377 km
33.7828125000:33.7838541665:132.786718750:132.786718750
50325642222
存在メッシュ：50325642222値1.95距離0.0 km
50325632444
50325632444 is kddi none!
33.7828125000:33.7838541665:132.786718750:132.786718750
50325642222
存在メッシュ：50325642222値2.025距離0.0 km
50325632444
50325632444 is kddi none!
33.7828125000:33.7838541665:132.786718750:132.786718750
50325642222
存在メッシュ：50325642222値2.206896551724138距離0.0 km
50325632444
50325632444 is kddi none!
33.7828125000:33.7838541665

33.7828125000:33.7838541665:132.786718750:132.786718750
50325642222
存在メッシュ：50325642222値1.95距離0.0 km
50325632444
存在メッシュ：50325632444値2.0227272727272725距離0.1155400634006377 km
33.7828125000:33.7838541665:132.786718750:132.786718750
50325642222
存在メッシュ：50325642222値2.025距離0.0 km
50325632444
存在メッシュ：50325632444値2.3523255813953483距離0.1155400634006377 km
33.7828125000:33.7838541665:132.786718750:132.786718750
50325642222
存在メッシュ：50325642222値2.206896551724138距離0.0 km
50325632444
50325632444 is kddi none!
33.7828125000:33.7838541665:132.786718750:132.786718750
50325642222
存在メッシュ：50325642222値1.7307692307692308距離0.0 km
50325632444
50325632444 is kddi none!
33.7828125000:33.7838541665:132.786718750:132.786718750
50325642222
存在メッシュ：50325642222値1.9545454545454546距離0.0 km
50325632444
50325632444 is kddi none!
33.7828125000:33.7838541665:132.786718750:132.786718750
50325642222
存在メッシュ：50325642222値2.0距離0.0 km
50325632444
50325632444 is kddi none!
33.7828125000:33.7838541665:132.786718750:132.786718750
50325

33.7828125000:33.7838541665:132.786718750:132.786718750
50325642222
存在メッシュ：50325642222値2.206896551724138距離0.0 km
50325632444
存在メッシュ：50325632444値2.6463512429831595距離0.1155400634006377 km
33.7828125000:33.7838541665:132.786718750:132.786718750
50325642222
存在メッシュ：50325642222値1.7307692307692308距離0.0 km
50325632444
存在メッシュ：50325632444値1.8186813186813189距離0.1155400634006377 km
33.7828125000:33.7838541665:132.786718750:132.786718750
50325642222
存在メッシュ：50325642222値1.9545454545454546距離0.0 km
50325632444
存在メッシュ：50325632444値2.1504702194357366距離0.1155400634006377 km
33.7828125000:33.7838541665:132.786718750:132.786718750
50325642222
存在メッシュ：50325642222値2.0距離0.0 km
50325632444
存在メッシュ：50325632444値2.928571428571429距離0.1155400634006377 km
33.7828125000:33.7838541665:132.786718750:132.786718750
50325642222
存在メッシュ：50325642222値1.4242424242424243距離0.0 km
50325632444
存在メッシュ：50325632444値1.5782145782145782距離0.1155400634006377 km
33.7828125000:33.7838541665:132.786718750:132.786718750
50325642222
存在メッシュ：5032564

In [363]:
connection.close()

In [358]:
#CreateGeojsonFile